In [ ]:
import warnings

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from math import ceil, sqrt
import random


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error

import torch
import torch.nn as nn

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and being used")
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU instead")

warnings.filterwarnings('ignore')
random.seed(12345)
np.random.seed(12345)
torch.manual_seed(12345)
#torch.use_deterministic_algorithms(True)
#Сброс ограничений на число столбцов
#pd.options.display.max_columns = None
RANDOM_STATE=12345

In [ ]:
import requests
TOKEN = "6015117129:AAGCdFJKD8mCoJc3CdMN-4b7ktiEbhpAsYA"
url = f"https://api.telegram.org/bot6015117129:AAGCdFJKD8mCoJc3CdMN-4b7ktiEbhpAsYA/getUpdates"
print(requests.get(url).json())

TOKEN = "6015117129:AAGCdFJKD8mCoJc3CdMN-4b7ktiEbhpAsYA"
chat_id = "916785354"
message = "Done!"
url = f"https://api.telegram.org/bot6015117129:AAGCdFJKD8mCoJc3CdMN-4b7ktiEbhpAsYA/sendMessage?chat_id=916785354&text=Done!"

## Загрузка данных

In [ ]:
#df = pd.read_excel('stars.xlsx', sheet_name='stars')


df = pd.read_csv('stars.csv', sep='\t')
#df = pd.read_csv('/datasets/6_class.csv')

## Предобработка и анализ данных

In [ ]:
df

In [ ]:
df.info()

Без пропусков - это прекрасно. Непонятная первая колонка - без названия, целочисленная.

In [ ]:
df.sample(8)

Первая колонка - это индекс, удалю. В последней вижу необходимость приведения слов к одинаковому написанию.

In [ ]:
#df = df.drop('Unnamed: 0', axis=1)

df = df.drop(['Unnamed: 0','Unnamed: 0.1'], axis=1)

In [ ]:
df.describe()

В этих данных категории в колонке 'Star type' и 'Star color'. Со 'Star type' будет коррелировать абсолютно всё, потому что типы как раз выделяются за счёт группировки объектов в кластеры, так можно сказать. И у каждого кластера определённые параметры.

Рассмотрим значения цветов. Вызывают сомнения 'yellowish' и сложные цвета. Оказывается, "желтоватый" вполне нормальный термин, а "Blue-white" и "Blue white" не одинаковые цвета в таблицах классификации (можно ознакомиться, например, здесь => https://kosmosgid.ru/zvyozdy/klassi-zvezd). Однако, у конкретной звезды должен быть один цвет, поэтому считаю уместным в двойных словах ставить тире. Нам дано, что этот параметр определён по спектру, а не по таблице классификации, поэтому должен быть однозначным. 

In [ ]:
df['Star color'].unique()

In [ ]:
star_color = {
'Red':'red', 
'Blue White':'blue-white', 
'White':'white', 
'Yellowish White':'yellowish-white', 
'Blue white':'blue-white',
'Pale yellow orange':'yellow-orange', 
'Blue':'blue', 
'Blue-white':'blue-white', 
'Whitish':'whitish',
'yellow-white':'yellow-white', 
'Orange':'orange', 
'White-Yellow':'yellow-white', 
'Blue ':'blue',
'Yellowish':'yellowish', 
'Orange-Red':'orange-red', 
'Blue white ':'blue-white',
'Blue-White':'blue-white' }  
df['Star color'] = df['Star color'].replace(star_color)
df['Star color'].unique()

In [ ]:
df.groupby('Star color').agg({'Temperature (K)':['min','max','count']})

В каких-то категориях будет маловато объектов, но таковы данные. 

In [ ]:
df.duplicated().sum()

Дубли отсутствуют.

In [ ]:
sns.pairplot(df, hue='Star type');

Между абсолютными температурой и звёздной величиной вырисовывается известная зависимость Герцшпрунга-Рассела(спектр/светимость) с главной последовательностью и областями гигантов и молодых звёзд. 
Радиус нам показывает, что в большинстве звёзды сравнимы с нашим Солнцем и только гипергиганты занимают большую площадь графиков (имеют сильный разброс значений радиуса). 
Также гипергиганты и сверхгиганты имеют отрицательную абсолютную звёздную величину и группируются в левой части графика.
В светимости тоже видим разброс значений для сверхгигантов и гипергигантов, но также и звёзды главной последовательности при высоких температурах могут демонстрировать разнообразие значений светимости. 

In [ ]:
columns = df.columns


In [ ]:
for i in columns[1:-1]:
    fig, (ax_hist, ax_box) = plt.subplots(2, sharex=True, gridspec_kw={"height_ratios": (.8, .2)}, figsize=(12, 8))

    sns.histplot(x=df[i], ax=ax_hist, color='palevioletred')
    sns.boxplot(x=df[i], ax=ax_box, color='palevioletred')

    ax_box.set(xlabel=i)
    ax_hist.set(ylabel='Count')
    ax_hist.set(title=i+' distribution')
    plt.show()


  

Все типы представлены равным количеством звёзд, и при этом их характеристики разнообразны.
Светимость в основном низкая с небольшим всплеском в первой четверти, радиус маленький, с промежутком между гигантами и остальными, а звёздная величина имеет два максимума.

Рассмотрим целевой признак.

In [ ]:

fig, (ax_hist, ax_box) = plt.subplots(2, sharex=True, gridspec_kw={"height_ratios": (.8, .2)}, figsize=(12, 8))

sns.histplot(x=df['Temperature (K)'], ax=ax_hist, color='palevioletred')
sns.boxplot(x=df['Temperature (K)'], ax=ax_box, color='palevioletred')

ax_box.set(xlabel='Temperature (K)')
ax_hist.set(ylabel='Count')
ax_hist.set(title='Temperature distribution')
plt.show()

Основная масса звёзд приходится на низкие температуры и медленное горение, что примерно соответствует ситуации в общем (http://www.astronet.ru/db/msg/1177040/chapter7_05.html - до 90% звёзд относятся к нормальным) . Особенные звёзды не будем отбрасывать - данных мало, и такие объекты действительно обнаружены.

Посмотрим, что показывает матрица корреляций.

In [ ]:
correlation_matrix = df.corr()

plt.figure(figsize= (16, 8))
sns.heatmap(correlation_matrix, annot = True)

Температура напрямую коррелирует со светимостью и типом звезды, и обратно коррелирует с абсолютной звёздной величиной.

Абсолютная звёздная величина показывает обратную зависимость от всех признаков, особенно сильно связана с типом звезды, и это действительно так.

Надо перевести значения колонки 'Star type' в строки, иначе дальше не хочет кодировщик обрабатывать числа как категории.

In [ ]:
df['Star type'] = df['Star type'].astype('str')

In [ ]:
df.info()

Теперь нужно разделить данные на таргет и фичи, на обучающую, тестовую и валидационную выборки.

Отделяю 30 процентов на тест и валидацию, остальное оставляю сети на обучение.

In [ ]:
X = df.drop('Temperature (K)', axis=1)
y = df['Temperature (K)']
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=0.7, random_state=RANDOM_STATE, shuffle=True)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, train_size=0.5, random_state=RANDOM_STATE, shuffle=True)

In [ ]:
what_words = ['Тренировочных признаков','Тренировочных целей', 'Тестовых признаков', 
        'Тестовых целей','Валидационных признаков', 'Валидационных целей']
what_arrs = [y_train, X_train, y_test, X_test, y_valid, X_valid]

for i in range(len(what_words)):
    print(f'{what_words[i]} {what_arrs[i].shape[0]*100/df.shape[0]:.0f}% от общей выборки.')

Теперь нужно закодировать категориальные данные и масштабировать численные.

In [ ]:
columns


In [ ]:
numcol = ['Luminosity(L/Lo)', 'Radius(R/Ro)',
       'Absolute magnitude(Mv)']
catcol = ['Star type', 'Star color']

OHE для категорий, StandartScaler для чисел.

In [ ]:
# .reset_index(inplace='True')

In [ ]:
ohe = OneHotEncoder(sparse=False, drop = 'first', handle_unknown = 'error')

In [ ]:
ohe = ohe.fit(df[catcol])

In [ ]:
#X_train.reset_index(inplace=True)


X_train_cat = pd.DataFrame(ohe.transform(X_train[catcol]))
X_train_cat.columns = ohe.get_feature_names_out(catcol)
X_train.drop(catcol, axis = 1, inplace = True)
X_train_cat.index=X_train.index
X_train = pd.concat([X_train, X_train_cat], axis = 1)

In [ ]:
X_test_cat = pd.DataFrame(ohe.transform(X_test[catcol]))
X_test_cat.columns = ohe.get_feature_names_out(catcol)
X_test.drop(catcol, axis = 1, inplace = True)
X_test_cat.index=X_test.index
X_test = pd.concat([X_test, X_test_cat], axis = 1)

In [ ]:
X_valid_cat = pd.DataFrame(ohe.transform(X_valid[catcol]))
X_valid_cat.columns = ohe.get_feature_names_out(catcol)
X_valid.drop(catcol, axis = 1, inplace = True)
X_valid_cat.index=X_valid.index
X_valid = pd.concat([X_valid, X_valid_cat], axis = 1)

In [ ]:
scaler = StandardScaler()

In [ ]:
X_train[numcol] = scaler.fit_transform(X_train[numcol])
X_test[numcol] = scaler.transform(X_test[numcol])
X_valid[numcol] = scaler.transform(X_valid[numcol])

Поскольку в нейронной сети используются тензоры, переведём данные в этот тип.

In [ ]:
X_train = torch.FloatTensor(X_train.values)
X_train = X_train.to(device)

y_train = torch.FloatTensor(y_train.values)
y_train = y_train.to(device)

X_test = torch.FloatTensor(X_test.values)
X_test = X_test.to(device)

y_test = torch.FloatTensor(y_test.values)
y_test = y_test.to(device)

X_valid = torch.FloatTensor(X_valid.values)
X_valid = X_valid.to(device)

y_valid = torch.FloatTensor(y_valid.values)
y_valid = y_valid.to(device)

In [ ]:
X_train

In [ ]:
y_train

Данные готовы, можно приступать к построению базовой сети.

## Построение базовой нейронной сети

In [ ]:
print('Входящих нейронов (параметров) сети будет', X_train.shape[1])

In [ ]:
n_in_neurons = X_train.shape[1]
h1 = [20, 50, 100, 300, 500, 800, 1000, 1200, 1500, 2000]
h2 = [20, 50, 100, 300, 500, 800, 1000, 1200, 1500, 2000]
h3 = [20, 50, 100, 300, 500, 800, 1000, 1200, 1500, 2000]
n_out_neurons = 1

 

Создаю подкласс от класса nn.Module и задаю функции создания и работы сети.

Подберём количество скрытых слоёв.

In [ ]:
class Model1(nn.Module):
    def __init__(self, n_in_neurons, n_hidden_neurons_1, n_out_neurons):
        super(Model1, self).__init__()
        
        self.fc1 = nn.Linear(n_in_neurons, n_hidden_neurons_1)
        self.act1 = nn.ReLU()
        self.fc2 = nn.Linear(n_hidden_neurons_1, n_out_neurons)
        
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)
        x = self.fc2(x)
        
        return x          



In [ ]:
class Model2(nn.Module):
    def __init__(self, n_in_neurons, n_hidden_neurons_1, n_hidden_neurons_2, n_out_neurons):
        super(Model2, self).__init__()
        
        self.fc1 = nn.Linear(n_in_neurons, n_hidden_neurons_1)
        self.act1 = nn.ReLU()
        self.fc2 = nn.Linear(n_hidden_neurons_1, n_hidden_neurons_2)
        self.act2 = nn.ReLU()
        self.fc3 = nn.Linear(n_hidden_neurons_2, n_out_neurons)

        
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)
        x = self.fc2(x)
        x = self.act2(x)
        x = self.fc3(x)
       
        return x          



In [ ]:
class Model3(nn.Module):
    def __init__(self, n_in_neurons, n_hidden_neurons_1, n_hidden_neurons_2, n_hidden_neurons_3, n_out_neurons):
        super(Model3, self).__init__()
        
        self.fc1 = nn.Linear(n_in_neurons, n_hidden_neurons_1)
        self.act1 = nn.ReLU()
        self.fc2 = nn.Linear(n_hidden_neurons_1, n_hidden_neurons_2)
        self.act2 = nn.ReLU()
        self.fc3 = nn.Linear(n_hidden_neurons_2, n_hidden_neurons_3)
        self.act3 = nn.ReLU()
        self.fc4 = nn.Linear(n_hidden_neurons_3, n_out_neurons)
        
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)
        x = self.fc2(x)
        x = self.act2(x)
        x = self.fc3(x)
        x = self.act3(x)
        x = self.fc4(x)
        #x = self.act4(x)
        return x          



Выбираю функцию потерь MSE и оптимизатор.

In [ ]:
#loss = nn.MSELoss()

def optimizer_make(net):
    optimizer1 = torch.optim.Adam(net.parameters(), lr=1e-3)
    optimizer2 = torch.optim.RMSprop(net.parameters(), lr=1e-3)
    return [optimizer1, optimizer2]

In [ ]:
def learn_model(net, optimizer, num_epochs, batch_size, X_train, y_train, X_test, y_test,):
    loss = nn.MSELoss()
    
    num_batches = ceil(len(X_train)/batch_size)
    
    for epoch in range(num_epochs):
        
        order = np.random.permutation(len(X_train)) # создайте случайную перестановку индексов объектов
        
        optimizer.zero_grad()
        
        for batch_idx in range(num_batches):
            start_index = batch_idx * batch_size
            # получение индексов текущего батча
            batch_indexes = order[start_index:start_index+batch_size]
            X_batch = X_train[batch_indexes]
            y_batch = y_train[batch_indexes]
    
            preds = net.forward(X_batch) 
                
            loss_value = loss(preds, y_batch)
            loss_value.backward()
            
            if ((batch_idx + 1) % accumulation_iteration == 0) or (batch_idx + 1 == num_batches):
                
                optimizer.step()
                optimizer.zero_grad()
                
        if epoch == num_epochs - 1:
            net.eval()
            test_preds = net.forward(X_test)
            RMSE = torch.sqrt(loss(preds, y_test))
        
            return torch.detach(RMSE).cpu().numpy()

In [ ]:
rmse1 = [['n_hidden_neurons_1', 'optimizer type', 'rmse']]
rmse2 = [['n_hidden_neurons_1', 'optimizer type', 'n_hidden_neurons_2', 'rmse']]
rmse3 = [['n_hidden_neurons_1', 'optimizer type', 'n_hidden_neurons_2', 'n_hidden_neurons_3', 'rmse']]
num_epochs = 200
batch_size = 40
accumulation_iteration = 5
for i in range(10):

    n_hidden_neurons_1 = h1[i]
    net1 = Model1(n_in_neurons, n_hidden_neurons_1, n_out_neurons)
    net1 = net1.to(device)
    optimizer_arr = optimizer_make(net1)
    for ind in range(len(optimizer_arr)): 
        optimizer = optimizer_arr[ind]
        rmse1.append([n_hidden_neurons_1, type(optimizer), 
            learn_model(net1, optimizer_arr[ind], num_epochs, batch_size, X_train, y_train, X_test, y_test,)])
        
    for j in range (10):
        if i*j % 15 == 0:
            n_hidden_neurons_2 = h2[j] 
    
            net2 = Model2(n_in_neurons, n_hidden_neurons_1, n_hidden_neurons_2, n_out_neurons)
            net2.to(device)
            optimizer_arr = optimizer_make(net2)    
            for ind in range(len(optimizer_arr)): 
                optimizer = optimizer_arr[ind]
                rmse2.append([n_hidden_neurons_1, n_hidden_neurons_2, type(optimizer),
                      learn_model(net2, optimizer_arr[ind], num_epochs, batch_size, X_train, y_train, X_test, y_test,)])
        for k in range(10):
            if i*j*k % 250 == 0:
                n_hidden_neurons_3 = h3[np.random.randint(0, len(h1))]

                net3 = Model3(n_in_neurons, n_hidden_neurons_1, n_hidden_neurons_2, n_hidden_neurons_3, n_out_neurons)
                net3.to(device)
                optimizer_arr = optimizer_make(net3)
                for ind in range(len(optimizer_arr)): 
                    optimizer = optimizer_arr[ind]                    
                    rmse3.append([n_hidden_neurons_1, n_hidden_neurons_2, n_hidden_neurons_3, type(optimizer),
                          learn_model(net3, optimizer_arr[ind], num_epochs, batch_size, X_train, y_train, X_test, y_test,)])
  


    #net1 = Model1(n_in_neurons, n_hidden_neurons_1, n_out_neurons)
    #net2 = Model2(n_in_neurons, n_hidden_neurons_1, n_hidden_neurons_2, n_out_neurons)
    #net3 = Model3(n_in_neurons, n_hidden_neurons_1, n_hidden_neurons_2, n_hidden_neurons_3, n_out_neurons)
    
            
    
    #rmse1.append([n_hidden_neurons_1, learn_model(net1, num_epochs, X_train, y_train, X_test, y_test,)])
    #rmse2.append([n_hidden_neurons_1, n_hidden_neurons_2, 
                  #learn_model(net2, num_epochs, X_train, y_train, X_test, y_test,)])
    #rmse3.append([n_hidden_neurons_1, n_hidden_neurons_2, n_hidden_neurons_3, 
                  #learn_model(net3, num_epochs, X_train, y_train, X_test, y_test,)])


In [ ]:
rmse1

In [ ]:
rmse2

In [ ]:
rmse3

Нейросеть с двумя скрытыми слоями показала динамику лучше, да вообще динамику, чем другие варианты. Оптимизатор Адам разгоняется дольше, но в конце выходит всё-таки на меньшую ошибку, чем RMSprop. Посмотрим, как она предсказывает температуру звезды на графике. обучим нейросеть на лучших на текущий момент параметрах. 

In [ ]:
n_hidden_neurons_1 = 2000
n_hidden_neurons_2 = 800
net = Model2(n_in_neurons, n_hidden_neurons_1, n_hidden_neurons_2, n_out_neurons)
net = net.to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)
loss = nn.MSELoss()
num_batches = ceil(len(X_train)/batch_size)
    
for epoch in range(num_epochs):
        
    order = np.random.permutation(len(X_train)) # создайте случайную перестановку индексов объектов
        
    optimizer.zero_grad()
        
    for batch_idx in range(num_batches):
        start_index = batch_idx * batch_size
        # получение индексов текущего батча
        batch_indexes = order[start_index:start_index+batch_size]
        X_batch = X_train[batch_indexes]
        y_batch = y_train[batch_indexes]
    
        preds = net.forward(X_batch) 
                
        loss_value = loss(preds, y_batch)
        loss_value.backward()
            
        if ((batch_idx + 1) % accumulation_iteration == 0) or (batch_idx + 1 == num_batches):
                
            optimizer.step()
            optimizer.zero_grad()
                
    if epoch == num_epochs - 1:
        net.eval()
        test_preds = net.forward(X_test)
        RMSE = torch.sqrt(loss(preds, y_test))
print(torch.detach(RMSE).cpu().numpy())        

In [ ]:
y1 = torch.FloatTensor(test_preds.cpu()).detach().numpy().reshape([-1])
y2 = y_test.detach().cpu().numpy().reshape([-1])
x = np.arange(len(y1))

fig,ax = plt.subplots()
fig.set_figwidth(18)
fig.set_figheight(8)
ax.set_xticks(x)
fact = ax.bar(x, y2, width = 0.6, label = 'Факт')
forecast = ax.bar(x, y1, width = 0.3, label = 'Прогноз')
ax.legend()
ax.set_title('Факт-прогноз', fontsize=20)
ax.set_xlabel('Номер звезды в таблице')
ax.set_ylabel('Температура звезды')
plt.show()

Как-то не очень, даже очень не. Давайте улучшим модель.

In [ ]:


del(rmse1, rmse2, rmse3, net1, optimizer_arr, i, j, k, ind, Model1, Model2, Model3)


## Улучшение нейронной сети

Введём регуляризацию двух типов - нормализация батчей и отключение нейронов.

In [ ]:
class Model(nn.Module):
    def __init__(self, n_in_neurons, n_hidden_neurons_1, n_hidden_neurons_2, n_out_neurons, p):
        super(Model, self).__init__()
        
        self.fc1 = nn.Linear(n_in_neurons, n_hidden_neurons_1)
        self.bn1 = nn.BatchNorm1d(n_hidden_neurons_1)
        self.act1 = nn.ReLU()
        self.dp2 = nn.Dropout(p=p)
        self.fc2 = nn.Linear(n_hidden_neurons_1, n_hidden_neurons_2)
        self.bn2 = nn.BatchNorm1d(n_hidden_neurons_2)
        self.act2 = nn.ReLU()
        self.fc3 = nn.Linear(n_hidden_neurons_2, n_out_neurons)
     
    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.act1(x)
        x = self.dp2(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.act2(x)
        x = self.fc3(x)
       
        return x     

Введём диапазоны параметров сети, которые попробуем перебрать для поиска лучшей модели и желаемой точности. (

In [ ]:
p_arr = [0.8, 0.5, 0.1, 0]
lr_arr = [0.01, 0.02, 0.025, 0.03, 0.035, 0.04]
batch_size_arr = [10,20,30,40,50,60,70,80,90,100,110,120,240]
num_epochs_arr = [1000, 1500, 2000, 10000]
logs = []

In [ ]:
def finding_neverland(p, lr, batch_size, num_epochs):
    
    net = Model(n_in_neurons, n_hidden_neurons_1, n_hidden_neurons_2, n_out_neurons, p)
    net = net.to(device)
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    loss = nn.MSELoss()
    num_batches = ceil(len(X_train)/batch_size)
    
    for epoch in range(num_epochs):
        
        order = np.random.permutation(len(X_train)) # создайте случайную перестановку индексов объектов
        
        optimizer.zero_grad()
        
        for batch_idx in range(num_batches):
            start_index = batch_idx * batch_size
            # получение индексов текущего батча
            batch_indexes = order[start_index:start_index+batch_size]
            X_batch = X_train[batch_indexes]
            y_batch = y_train[batch_indexes]
    
            preds = net.forward(X_batch) 
                
            loss_value = loss(preds, y_batch)
            loss_value.backward()
            
            if ((batch_idx + 1) % accumulation_iteration == 0) or (batch_idx + 1 == num_batches):
                
                optimizer.step()
                optimizer.zero_grad()
                
        if epoch == num_epochs - 1:
            net.eval()
            test_preds = net.forward(X_test)
            RMSE = torch.sqrt(loss(preds, y_test))
            return [torch.detach(RMSE).cpu().numpy(), p, lr, batch_size, num_epochs]
    

In [ ]:
for p in p_arr:
    for lr in lr_arr:
        for batch_size in batch_size_arr:
            for num_epochs in num_epochs_arr:
                logs.append(finding_neverland(p, lr, batch_size, num_epochs))
                
url = f"https://api.telegram.org/bot6015117129:AAGCdFJKD8mCoJc3CdMN-4b7ktiEbhpAsYA/sendMessage?chat_id=916785354&text=all_done"
print(requests.get(url).json())

In [ ]:
url = f"https://api.telegram.org/bot6015117129:AAGCdFJKD8mCoJc3CdMN-4b7ktiEbhpAsYA/sendMessage?chat_id=916785354&text=dataFrame"
print(requests.get(url).json())
pd.DataFrame(logs, columns=['RMSE', 'p', 'lr', 'batch_size', 'num_epochs'])

In [ ]:
y1 = torch.FloatTensor(test_preds.cpu()).detach().numpy().reshape([-1])
y2 = y_test.detach().cpu().numpy().reshape([-1])
x = np.arange(len(y1))

fig,ax = plt.subplots()
fig.set_figwidth(18)
fig.set_figheight(8)
ax.set_xticks(x)
fact = ax.bar(x, y2, width = 0.6, label = 'Факт')
forecast = ax.bar(x, y1, width = 0.3, label = 'Прогноз')
ax.legend()
ax.set_title('Факт-прогноз', fontsize=20)
ax.set_xlabel('Номер звезды в таблице')
ax.set_ylabel('Температура звезды')
plt.show()

In [ ]:
net_params = {
    'scaler' : scaler_list,
    'regressor__batch_size' : [10,20,30,40,50,60,70,80,90,100,110,120,240],
    'regressor__lr': [0.01, 0.02, 0.025, 0.03, 0.035, 0.04],
    'regerssor__max_epoch':[1000, 1500, 2000, 10000],
    'regressor__optimizer':[Adam, SGD, RMSprop],
    'regressor__module__dropout':[0.8, 0.5, 0.1, 0]
}

## Выводы